In [1]:
!pip3 install transformers[sentencepiece]

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
# Same as before
checkpoint = "papluca/xlm-roberta-base-language-detection"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [4]:
path = "./resources/raw/csv/"
df_all_artists_raw = pd.DataFrame()
for file in os.listdir(path):
    df_single_artist_raw = pd.read_csv(os.path.join(path, file))
    df_all_artists_raw = pd.concat([df_all_artists_raw, df_single_artist_raw])

df_all_artists_raw.tail()

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
59,59.0,Khalid,Khalid - Vertigo (Tradução Português),NaN,2018.0,2018-10-28,será que é melhor apenas acreditar nas teorias...
60,60.0,Khalid,Better (Miles Away Remix),NaN,2018.0,2018-12-12,i'm not really drunk i never get that fucked u...
61,61.0,Khalid,Khalid - Better (Official Music Video),NaN,2018.0,2018-05-07,users considering it's a virus or malware must...
62,62.0,Khalid,Perfect Lover,NaN,NaN,NaN,lyrics for this song have yet to be released p...
63,63.0,Khalid,Better (Rennie! Remix),NaN,2019.0,2019-02-01,love to see you shine in the night like the di...


In [7]:
path = './resources/intermediate'

if not os.path.exists(path):
  os.makedirs(path)

relevant_columns = ['Artist', 'Title', 'Lyric']
df_all_artists_raw.to_csv(os.path.join(path, 'df_all_artists_raw.csv'))

In [6]:
df_all_artists_lyrics_present = df_all_artists_raw[df_all_artists_raw.Lyric.notnull()]

In [8]:
sequences = list(df_all_artists_lyrics_present['Lyric'].str[:100])
sequences[:5]

["one one one one one   talkin' in my sleep at night makin' myself crazy out of my mind out of my mind",
 "if you don't wanna see me   did a full 80 crazy thinking 'bout the way i was did the heartbreak chan",
 "you call me all friendly tellin' me how much you miss me that's funny i guess you've heard my songs ",
 "i know it's hot i know we've got something that money can't buy fighting in fits biting your lip lov",
 "i see the moon i see the moon i see the moon oh when you're looking at the sun i'm not a fool i'm no"]

In [9]:
# sequences = [
#     "It's such a wonderful world.",
#     "Estoy muy contenta.",
# ]

model_inputs = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')
model_inputs

{'input_ids': tensor([[     0,   1632,   1632,  ...,      1,      1,      1],
        [     0,   2174,    398,  ...,      1,      1,      1],
        [     0,    398,  11782,  ...,      1,      1,      1],
        ...,
        [     0,  72095, 179635,  ...,      1,      1,      1],
        [     0,  68944,    100,  ...,      1,      1,      1],
        [     0,   5161,     47,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
output = model(**model_inputs)
output

In [ ]:
predictions = torch.nn.functional.softmax(output.logits, dim=-1)

In [ ]:

ranking = np.argmax(predictions.detach().numpy(), axis=1)
[model.config.id2label[i] for i in ranking]